In [1]:
import imapclient
import email

In [3]:
# support_tickets_password = input('password: ')

In [4]:
server = imapclient.IMAPClient(host='mail.archive.org', port='993')
server.login('support.tickets.bot@archive.org', support_tickets_password)

b'Logged in'

In [20]:
server.select_folder('UNMATCHED_ARCHIVE')
unmatched_archive_ids = server.search(['ALL', 'SENTSINCE', '05-DEC-2020'])

In [21]:
server.select_folder('MATCHED_ARCHIVE')
matched_archive_ids = server.search(['ALL', 'SENTSINCE', '05-DEC-2020'])

In [22]:
print('unmatched: ', len(unmatched_archive_ids))
print('matched: ', len(matched_archive_ids))

unmatched:  15222
matched:  4778


In [14]:
es = server.fetch(unmatched_archive_ids, ['ENVELOPE'])

abort: socket error: [Errno 32] Broken pipe

In [12]:
a = [1, 2, 3, 4]
a.remove(1)
a

[2, 3, 4]

In [245]:
from datetime import datetime, timedelta

In [261]:
data = server.fetch(ids[:1], ['ENVELOPE'])
data[ids[0]][b'ENVELOPE'].date

datetime.datetime(2020, 12, 1, 18, 33, 30)

# Difflib

In [1]:
import difflib

In [3]:
isjunk = lambda c: c in ' \t\n'

In [5]:
a = """
if you’re comparing lines as sequences of characters, and don’t want to synch up on blanks or hard tabs.

The optional arguments a and b are sequences to be compared; both default to empty strings. The elements of both sequences must be hashable.

The optional argument autojunk can be used to disable the automatic junk heuristic.

New in version 3.2: The autojunk parameter.

SequenceMatcher objects get three data attributes: bjunk is the set of elements of b for which isjunk is True; bpopular is the set of non-junk elements considered popular by the heuristic (if it is not disabled); b2j is a dict mapping the remaining elements of b to a list of positions where they occur. All three are reset whenever b is reset with set_seqs() or set_seq2().

New in version 3.2: The bjunk and bpopular attributes."""

b = """
if you’re comparing lines as sequences of characters, and don’t want to synch up on blanks or hard tabs.
\n
\nThe optional arguments a and b a sequences to be copared; both default to empty strings. The elements of both sequences must be hashable.
\n
The optional argument autojunk can be used to disable the automatic junk heuristic.
asd
\nNew in version 3.2: The autojunk parameter.

SequenceMatcher objects get three data attributes: bjunk is the set of elements of b for which isjunk is True; bpopular is the set of non-junk elements considered popular by the heuristic (if it is not disabled); b2j is a dict mapping the remaining elements of b to a list of positions where they occur. All three are reset whenever b is reset with set_seqs() or set_seq2().

New in version 3.2: The bjunk and bpopular attributes."""

In [6]:
m = difflib.SequenceMatcher(isjunk=isjunk, a=a, b=b, autojunk=False)

In [7]:
m

In [9]:
m.get_matching_blocks()

[Match(a=0, b=0, size=107),
 Match(a=107, b=109, size=32),
 Match(a=141, b=141, size=19),
 Match(a=161, b=160, size=88),
 Match(a=249, b=249, size=84),
 Match(a=333, b=337, size=476),
 Match(a=809, b=813, size=0)]

In [10]:
m.ratio()

0.9938347718865598

# Zendesk msgs hotswap

In [72]:
import requests
from requests.auth import HTTPBasicAuth
from pprint import pprint

In [73]:
ticket_num = 249426

In [76]:
# zd_password = input('ZD password: ')

In [77]:
r = requests.get(
    'https://archivesupport.zendesk.com/api/v2/tickets/{}/audits.json'.format(ticket_num),
    auth=HTTPBasicAuth('duncan@archive.org', zd_password)
)

In [78]:
first_audit_id = r.json()['audits'][0]['id']
first_audit_id

1269558196832

In [79]:
[e['type'] for e in r.json()['audits'][0]['events']]

['Comment', 'Create', 'Create', 'Create', 'Create', 'Create']

In [80]:
events = r.json()['audits'][0]['events']

In [176]:
for e in events:
    if e['type'].strip().lower() == 'comment':
        print(e['id'])

1269558197372


In [84]:
import re
p = re.compile('ZD(\d+):.*')
subject = 'ZD265958: Notification of payment received'
m = p.match(subject)
m.group(1)

'265958'

# Login

In [85]:
s = requests.Session()

In [86]:
r = s.get('https://archivesupport.zendesk.com/auth/v2/login/signin')

In [87]:
auth_token_pattern = re.compile(b'<input[^<^>]*?name="authenticity_token"[^<^>]*?value="(.*?)".*?/>')
m = auth_token_pattern.search(r.content)
auth_token = m.group(1).decode()
auth_token

'5w0zcmMK4jXDslpKHQlzJfaHMLjsPM0s4xBdGTMPYKuk8FYJTokWpRdsRlgXUqlA'

In [88]:
login_form_data = {
    'utf8': '✓',
    'authenticity_token': auth_token,
    'return_to_on_failure': '/auth/v2/login/signin',
    'return_to': 'https://help.archive.org/auth/v2/login/signed_in',
    'brand_id': '360000261412',
    'form_origin': 'no_return',
    'user[email]': 'duncan@archive.org',
    'user[password]': zd_password
}
r = s.post('https://archivesupport.zendesk.com/access/login', data=login_form_data)

In [89]:
r.status_code

200

# Get Raw Email

In [90]:
url = 'https://archivesupport.zendesk.com/audits/{}/email.eml?ticket_id={}'.format(first_audit_id, ticket_num)
url

'https://archivesupport.zendesk.com/audits/1269558196832/email.eml?ticket_id=249426'

In [91]:
r = s.get(url)
print(r.status_code)
if r.status_code != 200:
    print(r.content)

200


In [97]:
body = r.content

In [122]:
import io
zd_body_buf = io.StringIO(body.decode())

In [123]:
while zd_body_buf.readline().strip() != '':
    pass

In [124]:
zd_body_buf.readlines()

['<html><head><style>body=7Bfont-family:Helvetica,Arial;font-size:13px=7D</=\r\n',
 'style></head><body><div style=3D=22font-family:Helvetica,Arial;font-size:=\r\n',
 '13px; =22><br></div>This is my email that has a very important request<di=\r\n',
 'v><br></div><div>-Duncan<br><div class=3D=22gmail=5Fsignature=22 ></div><=\r\n',
 '/div></body></html>\r\n']

In [171]:
from collections import OrderedDict
d = OrderedDict({
    'this': 1,
    'that': 2
})

In [175]:
for k, v in d.items():
    print(k, v)

this 1
that 2


In [186]:
for i in enumerate(sorted(d.items(), key=lambda x: 1 - x[1])):
    print(i)

(0, ('that', 2))
(1, ('this', 1))


# what do with the rest?

In [168]:
import pickle
zd_unmatched, archive_unmatched = pickle.load(open('../src/temp.pickle', 'rb'))

In [169]:
zd_unmatched[0]

{b'SEQ': 1,
 b'INTERNALDATE': datetime.datetime(2020, 12, 1, 10, 48, 6),
 b'BODY[TEXT]': b'...\r\n',
 b'ENVELOPE': Envelope(date=datetime.datetime(2020, 12, 1, 10, 48, 3), subject=b'ZD267860: Notification of payment received', from_=(Address(name=b'Internet Archive', route=None, mailbox=b'support', host=b'archivesupport.zendesk.com'),), sender=(Address(name=b'Internet Archive', route=None, mailbox=b'support', host=b'archivesupport.zendesk.com'),), reply_to=(Address(name=b'Internet Archive', route=None, mailbox=b'support', host=b'archivesupport.zendesk.com'),), to=(Address(name=None, route=None, mailbox=b'support.tickets.bot', host=b'archive.org'),), cc=None, bcc=None, in_reply_to=None, message_id=b'<5fc665b398185_50b685b43133c_sprut@zendesk.com>')}

In [200]:
with open('../src/tmp/1.0a') as f:
    a = f.read()

In [202]:
with open('../src/tmp/1.0z') as f:
    z = f.read()

In [213]:
import difflib
import time

matcher = difflib.SequenceMatcher(isjunk=lambda c: c in ' \n\r\t')
matcher.set_seqs(a, z)

start = time.time()
r = matcher.real_quick_ratio()
print('quick ratio: {} in {} seconds'.format(r, time.time() - start))

start = time.time()
r = matcher.quick_ratio()
print('real quick ratio: {} in {} seconds'.format(r, time.time() - start))

start = time.time()
r = matcher.ratio()
print('ratio: {} in {} seconds'.format(r, time.time() - start))

quick ratio: 1.0 in 6.985664367675781e-05 seconds
real quick ratio: 1.0 in 0.016231298446655273 seconds
ratio: 1.0 in 6.669493198394775 seconds


In [222]:
with open('../src/tmp/0.5547861937734553a') as f:
    a = f.read()

In [223]:
with open('../src/tmp/0.5547861937734553z') as f:
    z = f.read()

In [217]:
import difflib
import time

matcher = difflib.SequenceMatcher(isjunk=lambda c: c in ' \n\r\t')
matcher.set_seqs(a, z)

start = time.time()
r = matcher.real_quick_ratio()
print('quick ratio: {} in {} seconds'.format(r, time.time() - start))

start = time.time()
r = matcher.quick_ratio()
print('real quick ratio: {} in {} seconds'.format(r, time.time() - start))

start = time.time()
r = matcher.ratio()
print('ratio: {} in {} seconds'.format(r, time.time() - start))

quick ratio: 0.9997410228703596 in 0.00011110305786132812 seconds
real quick ratio: 0.985006117218407 in 0.02104806900024414 seconds
ratio: 0.36544352066012376 in 6.2543768882751465 seconds


In [218]:
import diff_match_patch as dmp_module

In [229]:
dmp = dmp_module.diff_match_patch()
diff = dmp.diff_main(a, z)

In [230]:
dmp.diff_levenshtein(diff)

39436

1.0